In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import shutil
import random

categories = ["nothing", "other", "close", "left", "right", "ok"]
category_name_to_id = {k: str(v) for v, k in enumerate(categories)}

In [2]:
train_dir = "/hdd02/zhangyiyang/temporal-shift-module/data/AR/images/train2"
val_dir = "/hdd02/zhangyiyang/temporal-shift-module/data/AR/images/val2"

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
for category in categories:
    cur_dir = os.path.join(train_dir, category)
    if not os.path.exists(cur_dir):
        os.makedirs(cur_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
for category in categories:
    cur_dir = os.path.join(val_dir, category)
    if not os.path.exists(cur_dir):
        os.makedirs(cur_dir)

In [3]:
def _do_copy_imgs(source_dir, train_dir, val_dir, val_percent=.1, flip=False, save_every_n_steps=1, offset=0, shuffle=False):
    offset = offset % save_every_n_steps
    cnt = 0
    val_cnt = int(len(os.listdir(source_dir)) / save_every_n_steps * val_percent)
    source_files = os.listdir(source_dir)
    if flip:
        cc = os.path.basename(source_dir)
        if cc == 'left':
            train_dir = os.path.join(os.path.dirname(train_dir), 'right')
            val_dir = os.path.join(os.path.dirname(val_dir), 'right')
        elif cc == 'right':
            train_dir = os.path.join(os.path.dirname(train_dir), 'left')
            val_dir = os.path.join(os.path.dirname(val_dir), 'left')
    if shuffle:
        random.shuffle(source_files)
    cur_train_id = len(os.listdir(train_dir)) + 1
    cur_val_id = len(os.listdir(val_dir)) + 1
    cur_category = os.path.basename(train_dir)
    for idx, img_name in enumerate(source_files):
        if (idx+offset) % save_every_n_steps != 0:
            continue
        target_dir = val_dir if cnt < val_cnt else train_dir
        cur_id = cur_val_id if cnt < val_cnt else cur_train_id
        if flip:
            img = cv2.imread(os.path.join(source_dir, img_name))
            img = cv2.flip(img, flipCode=1)
            cv2.imwrite(os.path.join(target_dir, str(cur_id) + ".jpg"), img)
        else:
            shutil.copy(os.path.join(source_dir, img_name), os.path.join(target_dir, str(cur_id) + ".jpg"))
        if cnt < val_cnt :
            cur_val_id += 1
        else:
            cur_train_id += 1
        cnt += 1
    
    print('finish coping files from {}, get {}/{} train/val {} samples'.format(source_dir, cnt - val_cnt, val_cnt, cur_category))

In [4]:
filter_images_dir = "../../data/arimgs/openpose_results_manual_filter"
filter_save_every_n_steps_dict = {
    "nothing": 1,
    "other": 1,
    "close": 5,
    "left": 5,
    "right": 5,
    "ok": 5,
}
for category in categories:
    if not os.path.exists(os.path.join(filter_images_dir, category)):
        print(os.path.join(filter_images_dir, category), "doesn't exits")
        continue
    _do_copy_imgs(os.path.join(filter_images_dir, category), 
                  os.path.join(train_dir, category),
                  os.path.join(val_dir, category),
                  0.1,
                  save_every_n_steps=filter_save_every_n_steps_dict[category])
    _do_copy_imgs(os.path.join(filter_images_dir, category), 
                  os.path.join(train_dir, category),
                  os.path.join(val_dir, category),
                  0.1,
                  flip=True, offset=1,
                  save_every_n_steps=filter_save_every_n_steps_dict[category])
    

finish coping files from ../../data/arimgs/openpose_results_manual_filter/nothing, get 225/24 train/val nothing samples
finish coping files from ../../data/arimgs/openpose_results_manual_filter/nothing, get 225/24 train/val nothing samples
finish coping files from ../../data/arimgs/openpose_results_manual_filter/other, get 837/92 train/val other samples
finish coping files from ../../data/arimgs/openpose_results_manual_filter/other, get 837/92 train/val other samples
finish coping files from ../../data/arimgs/openpose_results_manual_filter/close, get 1045/116 train/val close samples
finish coping files from ../../data/arimgs/openpose_results_manual_filter/close, get 1044/116 train/val close samples
finish coping files from ../../data/arimgs/openpose_results_manual_filter/left, get 1110/123 train/val left samples
finish coping files from ../../data/arimgs/openpose_results_manual_filter/left, get 1109/123 train/val right samples
finish coping files from ../../data/arimgs/openpose_results

In [5]:
# background samples
to_label_summary_dir = "/hdd02/zhangyiyang/temporal-shift-module/data/AR/label/summary"
train_background_img_dir = os.path.join(train_dir, "nothing")
val_background_img_dir = os.path.join(val_dir, "nothing")
background_save_every_n_steps = 12
for to_file in os.listdir(to_label_summary_dir):
    if "background" in to_file and "flip" not in to_file:
        with open(os.path.join(to_label_summary_dir, to_file), "r") as f:
            samples = [line.strip() for line in f]
        for sample in samples:
            frames_dir = sample.split(" ")[0]
            _do_copy_imgs(frames_dir, 
                          train_background_img_dir, val_background_img_dir,
                          save_every_n_steps=background_save_every_n_steps)



finish coping files from /hdd03/zhangyiyang/data/AR/background/0601/frames/1, get 4/0 train/val nothing samples
finish coping files from /hdd03/zhangyiyang/data/AR/background/0601/frames/2, get 4/0 train/val nothing samples
finish coping files from /hdd03/zhangyiyang/data/AR/background/0601/frames/3, get 4/0 train/val nothing samples
finish coping files from /hdd03/zhangyiyang/data/AR/background/0601/frames/4, get 4/0 train/val nothing samples
finish coping files from /hdd03/zhangyiyang/data/AR/background/0601/frames/5, get 4/0 train/val nothing samples
finish coping files from /hdd03/zhangyiyang/data/AR/background/0601/frames/6, get 4/0 train/val nothing samples
finish coping files from /hdd03/zhangyiyang/data/AR/background/0601/frames/7, get 4/0 train/val nothing samples
finish coping files from /hdd03/zhangyiyang/data/AR/background/0601/frames/8, get 4/0 train/val nothing samples
finish coping files from /hdd03/zhangyiyang/data/AR/background/0601/frames/9, get 4/0 train/val nothing 

In [6]:
# EgoDexter
ego_dexter_dir = "../../data/arimgs/EgoDexter"
ego_dexter_save_every_n_steps = 15
train_other_imgs_dir = os.path.join(train_dir, "other")
val_other_imgs_dir = os.path.join(val_dir, "other")
for sub_dir in os.listdir(ego_dexter_dir):
    _do_copy_imgs(os.path.join(ego_dexter_dir, sub_dir), 
                  train_other_imgs_dir,
                  val_other_imgs_dir,
                  save_every_n_steps=ego_dexter_save_every_n_steps)
    _do_copy_imgs(os.path.join(ego_dexter_dir, sub_dir), 
                  train_other_imgs_dir,
                  val_other_imgs_dir,
                  save_every_n_steps=ego_dexter_save_every_n_steps,
                  offset=8, flip=True)

finish coping files from ../../data/arimgs/EgoDexter/Rotunda, get 87/9 train/val other samples
finish coping files from ../../data/arimgs/EgoDexter/Rotunda, get 87/9 train/val other samples
finish coping files from ../../data/arimgs/EgoDexter/Desk, get 31/3 train/val other samples
finish coping files from ../../data/arimgs/EgoDexter/Desk, get 30/3 train/val other samples
finish coping files from ../../data/arimgs/EgoDexter/Kitchen, get 31/3 train/val other samples
finish coping files from ../../data/arimgs/EgoDexter/Kitchen, get 30/3 train/val other samples


In [7]:
# FreiHand
freihand_dir = "../../data/arimgs/FreiHAND"
freihand_save_every_n_steps_dict = {
    "nothing": 1,
    "other": 2,
    "close": 1,
    "left": 1,
    "right": 1,
    "ok": 1,
}
for category in categories:
    if not os.path.exists(os.path.join(freihand_dir, category)):
        continue
    _do_copy_imgs(os.path.join(freihand_dir, category), 
                  os.path.join(train_dir, category),
                  os.path.join(val_dir, category),
                  save_every_n_steps=freihand_save_every_n_steps_dict[category])

finish coping files from ../../data/arimgs/FreiHAND/other, get 1644/182 train/val other samples
finish coping files from ../../data/arimgs/FreiHAND/close, get 39/4 train/val close samples
finish coping files from ../../data/arimgs/FreiHAND/left, get 1/0 train/val left samples
finish coping files from ../../data/arimgs/FreiHAND/right, get 1/0 train/val right samples
finish coping files from ../../data/arimgs/FreiHAND/ok, get 15/1 train/val ok samples


In [4]:
# NUS Hand Gesture v2
hus_dir = "../../data/arimgs/NUSV2"
hus_save_every_n_steps_dict = {
    "nothing": 5,
    "other": 5,
    "close": 5,
    "left": 5,
    "right": 5,
    "ok": 5,
}
for category in categories:
    if not os.path.exists(os.path.join(hus_dir, category)):
        continue
    _do_copy_imgs(os.path.join(hus_dir, category), 
                  os.path.join(train_dir, category),
                  os.path.join(val_dir, category),
                  save_every_n_steps=hus_save_every_n_steps_dict[category])
    _do_copy_imgs(os.path.join(hus_dir, category), 
                  os.path.join(train_dir, category),
                  os.path.join(val_dir, category),
                  save_every_n_steps=hus_save_every_n_steps_dict[category], flip=True, offset=2)

finish coping files from ../../data/arimgs/NUSV2/nothing, get 360/40 train/val nothing samples
finish coping files from ../../data/arimgs/NUSV2/nothing, get 360/40 train/val nothing samples
finish coping files from ../../data/arimgs/NUSV2/other, get 396/44 train/val other samples
finish coping files from ../../data/arimgs/NUSV2/other, get 396/44 train/val other samples
finish coping files from ../../data/arimgs/NUSV2/close, get 50/5 train/val close samples
finish coping files from ../../data/arimgs/NUSV2/close, get 50/5 train/val close samples
finish coping files from ../../data/arimgs/NUSV2/right, get 50/5 train/val right samples
finish coping files from ../../data/arimgs/NUSV2/right, get 50/5 train/val left samples


In [9]:
# places365 val
places365_val = "../../data/arimgs/places365_val"
train_background_img_dir = os.path.join(train_dir, "nothing")
val_background_img_dir = os.path.join(val_dir, "nothing")
places365_background_save_every_n_steps = 12
_do_copy_imgs(places365_val, 
              train_background_img_dir, val_background_img_dir,
              save_every_n_steps=places365_background_save_every_n_steps)



finish coping files from /hdd02/zhangyiyang/finetune_for_human/data/arimgs/places365_val, get 2738/304 train/val nothing samples


In [10]:
# summary
for category in categories:
    print(category, len(os.listdir(os.path.join(train_dir, category))))

nothing 9288
other 3614
close 2128
left 2087
right 2087
ok 2798


In [11]:
# summary
for category in categories:
    print(category, len(os.listdir(os.path.join(val_dir, category))))

nothing 352
other 396
close 236
left 231
right 231
ok 309
